In [2]:
import json

with open('../llm-zoomcamp/01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

948

In [3]:
len(documents)

948

Q1. Prepare Documents <br>
How many records we have in the pre-processed "documents"?

948


In [4]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"

from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Q2. Create Embeddings using Pretrained Models<br>
What is the model size (in MB) and average performance?<br>
<br>
[120, 59.76]

In [5]:
len(model.encode("This is a simple sentence"))

384

Q3. Get the dimension for model embedding<br>
What is the dimension of the model embedding?<br>

384

In [7]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)

In [12]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

try:
    es_info = es_client.info()
    print(es_info)
except Exception as e:
    print(f"Error: {e}")

es_client.info()

{'name': '3d934389e4b7', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'e-d_rMI7SWKM4t3S5QYYIg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


ObjectApiResponse({'name': '3d934389e4b7', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'e-d_rMI7SWKM4t3S5QYYIg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [14]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

Q4: Create Mappings and Index <br>
In the es_client.indices.delete statement, what does "ignore_unavailable" mean? <br>

If the index is unavailable, skip this statement

In [15]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'q-DHhZABMflKsFouCYsm',
  '_score': 0.852048,
  '_source': {'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}},
 {'_index': 'course-questions',
  '_id': 'zODHhZABMflKsFouDYso',
  '_score': 0.8381672,
  '_source': {'question': 'How can we contribute to the course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'}},
 {'_index': 'course-questions',
  '_id': 'buDHhZABMflKsFouOI3F',
  '_score': 0.8284229,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'Gene

Q5: Create end user query and perform semantic search <br>
What is the similarity score, section and course for the first result? <br>

[0.74, 'General course-related questions', 'data-engineering-zoomcamp']


In [17]:
knn_query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "bool": {
            "filter": {
                "term": {
                    "section": "General course-related questions"
                }
            }
        }
    }
)

response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'tODHhZABMflKsFouCotq',
  '_score': 0.0,
  '_source': {'text': 'It depends on your background and previous experience with modules. It is expected to require about 5 - 15 hours per week. [source1] [source2]\nYou can also calculate it yourself using this data and then update this answer.',
   'section': 'General course-related questions',
   'question': 'Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?',
   'course': 'data-engineering-zoomcamp',
   'question_vector': [0.06940636038780212,
    -0.030351007357239723,
    -0.011090363375842571,
    0.0906011313199997,
    0.009035035967826843,
    0.02179110050201416,
    -0.04435351863503456,
    -0.08035345375537872,
    -0.1470823585987091,
    0.003698865184560418,
    -0.022060610353946686,
    -0.01685948297381401,
    -0.08808361738920212,
    0.08680003136396408,
    -0.020892607048153877,
    -0.059946902096271515,
    0.029766183346509933,
    -0.1192118

Q6: Perform Semantic Search & Filtering <br>
Do you see the results filtered only to "General course-related questions" section? <br>

the original code is "No"
